# TFM Máster Data Science UAH 2020-2021

### MIGUEL PÉREZ CARO

Este notebook tiene como objetivo llevar a cabo el entrenamiento del modelo BERT para convertir el texto a vector a partir de este modelo dado que no se puede realizar en local por falta de medios.

Se comienza por importar las librerías necesarias

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 38.1 MB/s 
     |████████████████████████████████| 3.3 MB 18.5 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 67.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import time
import torch
import numpy as np
import pandas as pd
import transformers as ppb

Se lleva a cabo la importación del dataset

In [3]:
df = pd.read_csv('df_clean_no_dup.csv')

In [4]:
df.drop(columns=['id', 'created_at', 'retweet_count', 'favorite_count'], inplace = True)

In [5]:
df = df[df['text'].apply(lambda x: len(x.split(' ')) > 4)]
df.reset_index(drop=True, inplace=True)

In [6]:
df.sample(5)

,text,southwest,american,united,delta
26993,cmon know abby would rather skip go straight,0.0,0.0,0.0,1.0
147577,sent including screenshots siteapp errors,0.0,0.0,0.0,1.0
154335,southwest sent voucher raising hands mediumdar...,1.0,0.0,0.0,0.0
178683,united require us employees vaccinated later o...,0.0,0.0,1.0,0.0
71256,happy day avgeek airplanes boeing,0.0,1.0,1.0,0.0


In [7]:
n = 400  #chunk row size
df=df[98400:]
list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]

Se procede a trabajar con el modelo BERT

In [8]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Cargar model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
bert_array = np.zeros((0,768))

for iteration, item in enumerate(list_df):
  tokenized = item['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  # Obtener lista más larga
  max_len = len(max(tokenized, key=len))

  # Generar el array
  array_tokenized = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

  # Generar el attention_mas
  attention_mask = np.where(array_tokenized != 0, 1, 0)

  array_tokenized_input = torch.tensor(array_tokenized)  
  attention_mask = torch.tensor(attention_mask)

  with torch.no_grad():
    bert_corpus_full = model(array_tokenized_input, attention_mask=attention_mask)
  
  bert_vectors = bert_corpus_full[0][:,0,:].numpy()

  bert_corpus = np.zeros((len(bert_vectors), 768))

  for idx, vector in enumerate(bert_vectors):
    bert_corpus[idx, :] = vector
  bert_array = np.concatenate((bert_array, bert_corpus), axis = 0)
  
  if iteration%10==0:
    print(iteration)


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250


In [10]:
np.savetxt("bert_corpus_2.csv", bert_array, delimiter=",")


In [12]:
from google.colab import files
import pandas as pd
#result.to_csv('bert_corpus_2.csv')
files.download('bert_corpus_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>